# XoT Tutorial

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Prepare your OpenAI GPT Model

* Import OpenAI model with langchain. 
* You may run this part of code prior to the course to make sure that the model is ready.



In [4]:
import os
from dotenv import load_dotenv

import langchain
from langchain_openai import AzureChatOpenAI, ChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(
    SQLiteCache(database_path=".langchain_cache.db")
)  # setup cache for the LLM

langchain.debug = False  # set verbose mode to True to show more execution details

load_dotenv()

# # Use Azure's models
# langchain_llm = AzureChatOpenAI(
#     azure_endpoint=os.getenv("MY_AZURE_ENDPOINT"),
#     api_key=os.getenv("MY_AZURE_API_KEY"),
#     api_version=os.getenv("MY_AZURE_API_VERSION"),
#     azure_deployment=os.getenv("MY_AZURE_DEPLOYMENT_NAME"),
#     verbose=True,
# )

# Or use OpenAI's model
langchain_llm = ChatOpenAI(
    base_url=os.getenv("MY_OPENAI_API_BASE"),
    api_key=os.getenv("MY_OPENAI_API_KEY"),
    model="gpt-4o",
    verbose=True,
)

In [3]:
# a debug line to show you have the correct model setup
# If your set up is successful, it shows something like 
# AIMessage(content="LLM can refer to different ... 
langchain_llm.invoke("What is LLM?")

AIMessage(content='The term "LLM" can refer to different things depending on the context:\n\n1. **Master of Laws (LL.M.):** This is an advanced, postgraduate academic degree in law. It is often pursued by individuals who have already obtained a primary law degree (such as a J.D. or an LL.B.) and wish to gain specialized knowledge in a specific area of law, such as international law, human rights law, or tax law.\n\n2. **Large Language Model:** In the context of artificial intelligence and natural language processing, an LLM refers to a type of AI model designed to understand and generate human language. These models are trained on vast datasets containing diverse text sources and can perform a variety of language-related tasks, such as text completion, translation, summarization, and answering questions. Examples include OpenAI\'s GPT-3 and GPT-4.\n\n3. **Linked List Manager (LLM):** In computer science, LLM can also refer to software or a component within an operating system that mana

## Direct Use of LLM

* Practice the use of LLM with langchain.
* Intent classifier: does the user want to have its tablet repaired?

In [10]:
from utils.intent_classifier import intent_classifier_prompt, intent_classifier_parser

intent_classifier_chain = (
    intent_classifier_prompt | langchain_llm | intent_classifier_parser
)

In [11]:
prompt = intent_classifier_prompt.invoke({"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."})
output = langchain_llm.invoke(prompt)
intent_classifier_parser.invoke(output)

{'want_to_repair': 'Yes',
 'utterance': "Sure, I'd be happy to help you with your iPad's Bluetooth connection issue. Let's go through some troubleshooting steps."}

In [8]:
intent_classifier_chain.invoke(
    {"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."}
)

{'want_to_repair': 'Yes',
 'utterance': "I'm sorry to hear that your iPad is having trouble connecting to Bluetooth. Let me help you with that."}

## Chain of Thoughts Prompting

* Practice the use of CoT with langchain.
* Use warranty checker as an example.

In [5]:
from utils.warranty_module import warranty_parser, warranty_checker_prompt

warranty_checker_chain = warranty_checker_prompt | langchain_llm

In [8]:
warranty_checker_prompt.invoke({"user_input": "I want to check my warranty status."}).to_messages()

[SystemMessage(content="You are a helpful agent that help to decide if a product is out of warranty. Note that all product come with a 90-day warranty since purchase. Customer can also purchase additional warranty that extends it to 2 years. Today is May 20th, 2024. Reply if the product has warranty or not. If the product has warranty, reply 'Yes'. If the product does not have warranty, reply 'No'. If you are unsure, reply 'Unsure'."),
 HumanMessage(content='A month ago'),
 AIMessage(content="Analysis: Let's think step by step. All products come with a 90-day warranty since purchase. A month is within the 90-day time span. The product must have warranty. Warranty: Yes. Utterance: Great! Your product has warranty."),
 HumanMessage(content='Last year.'),
 AIMessage(content="Analysis: Let's think step by step. All products come with a 90-day warranty. A year is more than this time span. However, we are not sure if the user purchased additional warranty. We are not sure if the product has 

In [6]:
output = warranty_checker_chain.invoke({"user_input": "I bought this item on May 1st, 2023."})
warranty_parser.invoke(output)

{'Analysis': 'The user is unsure about the warranty status and has not provided a specific purchase date. Without this information, it is impossible to determine whether the product is still under warranty.',
 'Warranty': 'Unsure',
 'Utterance': 'I am sorry to hear that. To help you check your warranty status, could you provide me with the date of purchase or any information about an extended warranty?'}

## Self-Consistency with CoT

* We use self-consistent CoT to practice repair plan suggestion. Available plans are repair, replace and trade-in options. 
* Three CoT and a final evaluator. Following the structures below:


<center><img src="doc/figs/XoT_Tutorial_Diagrams-CoT-SC.svg" alt="CoT-SC Structure" style="width: 400px;"/></center>

In [17]:
from utils.repair_strategy import repair_strategy_prompt, repair_parser
from utils.replace_strategy import replacement_strategy_prompt, replacement_parser
from utils.tradein_strategy import tradein_strategy_prompt, tradein_parser

repair_strategy_chain = repair_strategy_prompt | langchain_llm
replacement_strategy_chain = replacement_strategy_prompt | langchain_llm
tradein_strategy_chain = tradein_strategy_prompt | langchain_llm

In [15]:
repair_strategy_chain.invoke(
    {"user_input": "I want to repair my device. The screen and the battery are broken."}
)
repair_parser.invoke(
    repair_strategy_chain.invoke(
        {
            "user_input": "I want to repair my device. The screen and the battery are broken."
        }
    )
)

{'Analysis': "Let's think step by step. The cost for fixing the screen is 120 USD, and the cost for fixing the battery is 30 USD. The total cost is 150 USD.",
 'Cost': '150 USD',
 'Utterance': 'The total cost to fix the product is 150 USD.'}

In [16]:
tradein_strategy_chain.invoke(
    {"user_input": "I want to trade in my device. The keyboard is not working"}
)
tradein_parser.invoke(
    tradein_strategy_chain.invoke(
        {"user_input": "I want to trade in my device. The keyboard is not working"}
    )
)

{'Analysis': 'The keyboard is not working, which is a significant issue. Trading in the device with a non-functional keyboard will still provide some value.',
 'Coupon': '100 USD',
 'Utterance': 'You can get a 100 USD coupon if you choose to trade in your old device for a new one.'}

In [13]:
replacement_strategy_chain.invoke({"user_input": "The screen is broken, I want to replace it.", "additional_info": "No warranty"})
replacement_parser.invoke(replacement_strategy_chain.invoke({"user_input": "The screen is broken, I want to replace it.", "additional_info": "No warranty"}))

{'Analysis': 'The user does not have a warranty, so they cannot get a replacement for the broken screen.',
 'Valid': 'No',
 'Utterance': "I'm sorry, but you cannot get a replacement for the screen. Only products with warranty can be replaced."}

In [13]:
from utils.plan_selection import plan_selection_prompt, plan_parser

plan_evaluater_chain = plan_selection_prompt | langchain_llm

In [19]:
plan_evaluater_chain.invoke(
    {
        "replace_plan": "I'm sorry, but you cannot get a replacement. Only products with warranty can be replaced.",
        "repair_plan": "The total cost to fix the product is 150 USD.",
        "tradein_plan": "You can get a 100 USD coupon if you want to trade in your old device for a new one.",
        "user_preference": "None",
        "selected_plan": "repair",
        "ai_analysis": "Let's thing step by step. The cost for a new device is 249 USD, the user can get a coupon so he will spend 149 USD for the new device. The cost for fixing the old device is 150 USD. Therefore, it is better to buy a new device as the cost is almost the same.",
        "ai_response": "Maybe you should consider buying a new device instead of fixing the old one. The cost for fixing the old device is 150 USD, and the cost for a new device is 249 USD. You can get a 100 USD coupon if you want to trade in your old device for a new one. The total cost to fix the product is 150 USD.",
    }
)
plan_parser.invoke(
    plan_evaluater_chain.invoke(
        {
            "replace_plan": "Your device can be replaced.",
            "repair_plan": "The total cost to fix the product is 150 USD.",
            "tradein_plan": "You can get a 100 USD coupon if you want to trade in your old device for a new one.",
            "user_preference": "None",
        }
    )
)

{'Analysis': 'The product can be replaced, repaired, or traded in. For repair, the cost is 150 USD. For trade-in, the price for a new device is 249 USD, and you can receive a 100 USD coupon, effectively spending 149 USD. Since the replacement is valid and costs 0 USD, it is the best and most cost-effective option.',
 'Cost': '0 USD',
 'Plan': 'replace',
 'Utterance': 'Your device can be replaced at no cost. This is the most cost-effective option compared to repairing, which would cost 150 USD, or trading in, which would cost you 149 USD for a new device after applying the 100 USD coupon.'}

In [18]:
from langchain_core.runnables import RunnableParallel

# The tot structure
repair_strategy_chain = repair_strategy_prompt | langchain_llm | repair_parser
replacement_strategy_chain = (
    replacement_strategy_prompt | langchain_llm | replacement_parser
)
tradein_strategy_chain = tradein_strategy_prompt | langchain_llm | tradein_parser

tot_generater = RunnableParallel(repair_plan = repair_strategy_chain, replace_plan = replacement_strategy_chain, tradein_plan = tradein_strategy_chain)

plan_output = tot_generater.invoke({
    "user_input": "The screen is broken, I want to replace it.",
    "additional_info": "No warranty"
})

tot_scorer = plan_selection_prompt | langchain_llm | plan_parser

tot_scorer.invoke({
    "replace_plan": plan_output["replace_plan"]["Utterance"],
    "repair_plan": plan_output["repair_plan"]["Utterance"],
    "tradein_plan": plan_output["tradein_plan"]["Utterance"],
    "user_preference": "None",
})

{'Analysis': 'The product cannot be replaced without a warranty. For repair, the cost to fix the screen is 120 USD. For trade-in, the cost for a new device is 249 USD, and the user can get a 100 USD coupon, so they will spend 149 USD for the new device. Therefore, it is better to buy a new device as the cost is almost the same.',
 'Cost': '149 USD',
 'Plan': 'Trade-in',
 'Utterance': 'Maybe you should consider buying a new device instead of fixing the old one. The cost to repair the screen is 120 USD, and the cost for a new device is 249 USD. You can get a 100 USD coupon if you want to trade in your old device for a new one, bringing the total cost to 149 USD.'}

## Tree of Thought

* More complicated repairing scenario. Add time and budget constrain to the repairing process.

<center><img src="doc/figs/XoT_Tutorial_Diagrams-ToT.svg" alt="ToT Structure" style="width: 400;"/></center>

In [17]:
plan_output_2 = tot_generater.invoke(
    {
        "user_input": "The screen is broken, I want to replace it.",
        "additional_info": "I have warranty.",
    }
)
first_layer_plan = tot_scorer.invoke(
    {
        "replace_plan": plan_output_2["replace_plan"]["Utterance"],
        "repair_plan": plan_output_2["repair_plan"]["Utterance"],
        "tradein_plan": plan_output_2["tradein_plan"]["Utterance"],
        "user_preference": "None",
    }
)
display(first_layer_plan)

{'Analysis': 'The product is under warranty, so the user can get a free replacement for the broken screen. The cost to repair the device is 120 USD, and the cost for a new device is 249 USD with a 100 USD coupon available for trade-in. Since the product is under warranty, the replacement is the best option as it costs 0 USD.',
 'Cost': '0 USD',
 'Plan': 'Replace',
 'Utterance': 'Your product is under warranty, so you can get a free replacement for the broken screen. This is the best option since it will cost you nothing.'}

In [18]:
from utils.timebased_generator import get_time_contrain_of_plans, time_aware_plan_parser, time_aware_plan_selection_prompt


time_aware_tot_scorer = time_aware_plan_selection_prompt | langchain_llm | time_aware_plan_parser

second_layer_plan = get_time_contrain_of_plans(first_layer_plan)

time_aware_tot_scorer.invoke({
    "plans": second_layer_plan,
})


{'Analysis': 'The product is under warranty, so the user can get a free replacement for the broken screen. The cost to repair the device is 120 USD, and the cost for a new device is 249 USD with a 100 USD coupon available for trade-in. Since the product is under warranty, the replacement is the best option as it costs 0 USD.',
 'Cost': '0 USD',
 'Time': 'Get a space grey tablet now as it is in stock.',
 'Plan': 'Replace',
 'Utterance': 'Your product is under warranty, so you can get a free replacement for the broken screen. This is the best option since it will cost you nothing. You can get a space grey tablet now as it is in stock.'}

## GoT structure

* We can reuse all the previous chains to form a GoT structure.
* Before that, we need to introduce two extra chain, one for collecting tech issue, one for generating questions for collecting tech issues from users.
* GoT structure is shown below:

<center><img src="doc/figs/XoT_Tutorial_Diagrams-GoT.svg" alt="GoT Structure" style="width: 400;"/></center>


In [19]:
# upon receiving user input, all nodes will update its internal state
from utils.techissue_analyser import tech_issue_analyser_prompt, tech_issue_analyser_parser

tech_issue_analyser_chain = tech_issue_analyser_prompt | langchain_llm | tech_issue_analyser_parser

In [20]:
tech_issue_analyser_chain.invoke(
    {
        "chat_history": "I bought this item on May 1st, 2023. \n The screen is broken, I want to replace it. \n It does not charge at the moment",
        "user_input": "Please help."
    }
)

{'tech_issue': ['Screen', 'Charging port']}

In [21]:
warranty_checker_fullchain = warranty_checker_prompt | langchain_llm | warranty_parser
warranty_checker_fullchain.invoke({"user_in put": "I'm not sure"})

{'Analysis': 'The user is unsure about the warranty. Without knowing the purchase date or whether additional warranty was bought, we cannot determine if the product is still under warranty.',
 'Warranty': 'Unsure',
 'Utterance': "I understand that you're unsure. Could you check any purchase receipts or emails to find out when the product was purchased? That information will help determine the warranty status."}

In [22]:
# A simple GoT Prompter: decide what question to ask based on the required information.
from utils.question_generator import (
    question_generator_prompt,
    question_generator_parser,
)  # The parser

question_generator_chain = (
    question_generator_prompt | langchain_llm | question_generator_parser
)

question_generator_chain.invoke(
    {
        "chat_history": "",
        "required_information": "preference (how much repair fee can afford?)",
        "suggested_question": "",
    }
)

question_generator_chain.invoke(
    {
        "chat_history": "",
        "required_information": "if user's product have warranty",
        "suggested_question": "I understand that you're unsure. Could you check any purchase receipts or emails to find out when the product was purchased? That information will help determine the warranty status.",
    }
)

# A simple GoT Controller: ask twice for the required information, and then generate a plan
# See the chatbot

{'utterance': "I understand that you're unsure. Could you check any purchase receipts or emails to find out when the product was purchased? That information will help determine the warranty status."}

In [23]:
from chatbot import Chatbot

customer_chatbot = Chatbot()

In [24]:
customer_chatbot.interact("hello")

'Hello! This is repairing bot. I can help you diagnose your tablet and provide you a repairing suggestion. How can I help you today?'

In [25]:
customer_chatbot.interact("I want to repair my tablet")

'Could you please provide me more information about the issue of your tablet? And when did you purchased this item?'

In [26]:
customer_chatbot.interact("Yeah, the screen is always black and there is no audio.")

'I am sorry to hear that. To determine if your product is still under warranty, could you please provide the purchase date?'

In [27]:
customer_chatbot.interact("Sorry I don't know")

'I am sorry to hear that. But I need to know if the product has warranty. Could you provide me the time of purchase?'

In [28]:
customer_chatbot.interact("I'm not sure")

"Based on the information you provided, I can generate a rought plan for you. It may take a few seconds. If you want me to generate a plan for you, please reply 'Generate a plan'. If you want to end the conversation, please reply 'Exit'."

In [29]:
customer_chatbot.interact("Generate a plan")

'Maybe you should consider buying a new device instead of fixing the old one. The cost for fixing the old device is 180 USD, and the cost for a new device is 249 USD. You can get a 100 USD coupon if you want to trade in your old device for a new one. The total cost for the new device after the trade-in will be 149 USD. You can get a space grey tablet now as it is in stock.'